# add c_EC as one of the inputs for diffusivity and conductivity

In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp,tanh;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False

In [2]:
# define function:
def electrolyte_conductivity_Nyman2008_wEC(c_e,c_EC, T):
    x = c_EC / c_e
    coff = 0.99
    sigma_e = (
        (0.01+ coff/2 + coff/2 *  tanh((x-2.7705)*2)) * 
        (0.1297 * (c_e / 1000) ** 3 - 2.51 * (c_e / 1000) ** 1.5 + 3.329 * (c_e / 1000)) )
    return sigma_e
def electrolyte_diffusivity_Nyman2008_wEC(c_e, c_EC,T):
    x = c_EC / c_e
    coff = 0.99
    D_c_e = (
        ( 0.01+ coff/2 + coff/2 *  tanh((x-2.7705)*2) )  * 
        (8.794e-11 * (c_e / 1000) ** 2 - 3.972e-10 * (c_e / 1000) + 4.862e-10) )
    return D_c_e


In [3]:
V_max = 4.2;        V_min = 2.5;
Diff_SEI = 1.7e-20;    R_SEI = 2E5;   Bulk_Sol_Con = 4541.0;
Experiment_Long   = pybamm.Experiment( [ (
    f"Discharge at 1 C until {V_min} V", 
    f"Charge at 0.3 C until {V_max} V", 
    f"Hold at {V_max} V until C/5")    ] * 1 ) 
Experiment_s   = pybamm.Experiment( [ (
    f"Discharge at 1 C for 197 s", )    ] * 1 )   

In [4]:
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P3R4/'; 
Target  = 'a2_Diff_Cond_c_EC/'
if not os.path.exists(BasicPath + Target):
    os.mkdir(BasicPath + Target);

In [5]:
Model_ddiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "EC wo refill"     } )
Model_sdiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "none"     } )
ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008_rio";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);


Para_0['EC Lithium ion cross diffusivity [m2.s-1]'] = 1.5e-9/1e2;
Para_0['Typical EC Lithium ion cross diffusivity [m2.s-1]'] =   1.5e-9/1e2
Para_0['EC transference number'] =    -1.4
Para_0['EC diffusivity in electrolyte [m2.s-1]'] =  5E-12;
Para_0.update(
    {'Electrolyte diffusivity [m2.s-1]':
    electrolyte_diffusivity_Nyman2008_wEC})
Para_0.update(
    {'Electrolyte conductivity [S.m-1]':
    electrolyte_conductivity_Nyman2008_wEC})
c_e_ddiff = Model_ddiff.variables["Electrolyte concentration [mol.m-3]"]
T_ddiff = Model_ddiff.variables["Cell temperature [K]"]
c_EC_ddiff = Model_ddiff.variables["EC concentration [mol.m-3]"]
Model_ddiff.variables["c(EC) over c(Li+)"] = c_EC_ddiff / c_e_ddiff
Model_ddiff.variables["Electrolyte conductivity [S.m-1]"] =electrolyte_conductivity_Nyman2008_wEC(c_e_ddiff,c_EC_ddiff, T_ddiff)
Model_ddiff.variables["Electrolyte diffusivity [m2.s-1]"] =electrolyte_diffusivity_Nyman2008_wEC(c_e_ddiff,c_EC_ddiff, T_ddiff)

using EC wo refill for Li+


In [6]:
Sim_Landes_ddiff    = pybamm.Simulation(
    Model_ddiff, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)   
Sim_Landes_sdiff    = pybamm.Simulation(
    Model_sdiff, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)  
Sol_Landes_ddiff    = Sim_Landes_ddiff.solve();
Sol_Landes_sdiff    = Sim_Landes_sdiff.solve();

In [7]:
Sol_Landes_ddiff["Electrolyte conductivity [S.m-1]"].entries[:,-1] 

array([0.94941452, 0.94943932, 0.94948519, 0.94954469, 0.94960662,
       0.94965612, 0.94967464, 0.94964011, 0.94952703, 0.94930664,
       0.94894648, 0.9484083 , 0.94763914, 0.94654224, 0.94489679,
       0.94218652, 0.93737283, 0.92892946, 0.91577505, 0.05642542,
       0.01128369, 0.9460046 , 0.9460771 , 0.94614884, 0.9462198 ,
       0.94629001, 0.94635944, 0.94642811, 0.94649601, 0.94656315,
       0.94662951, 0.94669511, 0.94675995, 0.94682402, 0.94688732,
       0.94694986, 0.94701163, 0.94707265, 0.9471329 , 0.94719239,
       0.94752054, 0.94803295, 0.94843761, 0.94871002, 0.9487811 ,
       0.94847089, 0.94733267, 0.94432898, 0.93724173, 0.9218269 ,
       0.89118784, 0.83671535, 0.75222542, 0.64011963, 0.51342453,
       0.38952485, 0.28147579, 0.1950195 , 0.13162132, 0.09449492])

In [11]:
font = {'family' : 'Times New Roman',
        #'weight' : 'bold',
        'size'   : 18}
mpl.rc('font', **font)

In [12]:
label = ["Landes_ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    #"Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    #"Loss of capacity to SEI [A.h]",
    "Li+ flux [mol.m-2.s-1]",
    "EC flux [mol.m-2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_Landes_ddiff
    ], output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=2,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
quick_plot.create_gif(
    number_of_images=10, duration=2,output_filename=BasicPath + Target+"fixed_add c_EC as tanh (0.01-1).gif")

interactive(children=(FloatSlider(value=0.0, description='t', max=0.18996945098811874, step=0.0018996945098811…

In [13]:
label = ["Landes_ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    #"Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    #"Loss of capacity to SEI [A.h]",
    "Li+ flux [mol.m-2.s-1]",
    "EC flux [mol.m-2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_Landes_ddiff
    ], output_variables3,label,
    variable_limits='tight',time_unit='hours',n_rows=2,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
quick_plot.create_gif(
    number_of_images=10, duration=2,output_filename=BasicPath + Target+"add c_EC as tanh (0.01-1).gif")

interactive(children=(FloatSlider(value=0.0, description='t', max=0.18996945098811874, step=0.0018996945098811…